# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino<br>
Spring 2025<br>

Name: Thompson Morgan
<br>
Date: 4/24/25
<br>
<br>
**At this time in the semester:** <br>
- We have explored a dataset. <br>
- We have cleaned our dataset. <br>
- We created a Github account with a repository for this class and included a metadata read me file about our data. <br>
- We introduced general SQL syntax, queries, and applications in Python.<br>
- Created our own databases from scratch using MySQL Workbench and Python with SQLAlchemy on our local server and locally on our machine.
- Populated our databases with the data we cleaned at the start of the semester.
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source, especially when posting your assignments to Github.<br>

Feel free to use your preferred library and method for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook!<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. Please submit this assignment with a link to it posted to your Github.<br>

**Data origin:** <br>
I am using the murder mystery database provided in Assignment #4 and #5. This database was created by @NUKnightLab on Github and can be found here: https://github.com/NUKnightLab/sql-mysteries

In [2]:
# Load necessary packages:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, BigInteger, Float, text, inspect # Database navigation
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import mysql.connector
import sqlite3 # A second option for working with databases
import pandas as pd # Python data manilpulation
import numpy as np # Populating our tables

In [4]:
# Connect to our .db file using library SQLAlchemy
db_path = "sql-murder-mystery.db"
engine = create_engine(f"sqlite:///{db_path}") # Use the create_engine function to connect to the database

In [6]:
# Write a query to list the table names of the database:
inspector = inspect(engine)  # Create an inspector to examine the database
tables = inspector.get_table_names()  # Get all table names from the database
print("Tables in the database:", tables)

Tables in the database: ['crime_scene_report', 'drivers_license', 'facebook_event_checkin', 'get_fit_now_check_in', 'get_fit_now_member', 'income', 'interview', 'person', 'solution']


#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** How many people in the database are 4' something, 5' something, or 6' something tall?
<br>
**What tables are we joining? (If any):** None

In [8]:
with engine.connect() as connection: # Establish a connection
    case_one = text("""SELECT COUNT(*) AS count,
                       CASE
                               WHEN height >= 84 THEN '7 feet tall or over'
                               WHEN height >= 72 THEN 'Between 6 and 7 feet tall'
                               WHEN height >= 60 THEN 'Between 5 and 6 feet tall'
                               WHEN height >= 48 THEN 'Between 4 and 5 feet tall'
                               ELSE 'Under 4 feet tall'
                           END AS height_group
                       FROM drivers_license
                       GROUP BY height_group
                       ORDER BY count DESC
                       """) # Define the query - text() ensures that the query string is read as a SQL expression
    case_one = pd.read_sql(case_one, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
case_one

,count,height_group
0,3613,Between 5 and 6 feet tall
1,3389,Between 6 and 7 feet tall
2,3004,Between 4 and 5 feet tall
3,1,7 feet tall or over


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** How many people checked in to the gym before, after, and on the day of the murder we previously solved?
<br>
**What tables are we joining? (If any):** None

In [10]:
with engine.connect() as connection: # Establish a connection
    case_two = text("""SELECT COUNT(*) AS count,
                       CASE
                               WHEN check_in_date > 20180115 THEN 'After the murder'
                               WHEN check_in_date = 20180115 THEN 'The day of the murder'
                               ELSE 'Before the murder'
                           END AS days_group
                       FROM get_fit_now_check_in
                       GROUP BY days_group
                       ORDER BY count DESC
                       """) # Define the query - text() ensures that the query string is read as a SQL expression
    case_two = pd.read_sql(case_two, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
case_two

,count,days_group
0,2112,Before the murder
1,587,After the murder
2,4,The day of the murder


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** How many people in the database make either 5, 6, or 7 figures for their annual income?
<br>
**What tables are we joining? (If any):** None

In [12]:
with engine.connect() as connection: # Establish a connection
    case_three = text("""SELECT COUNT(*) AS count,
                         CASE
                                 WHEN annual_income >= 1000000 THEN '7 Figures'
                                 WHEN annual_income >= 100000 THEN '6 Figures'
                                 WHEN annual_income >= 10000 THEN '5 Figures'
                                 ELSE '4 Figures or less'
                             END AS salary_group
                         FROM income
                         GROUP BY salary_group
                         ORDER BY count DESC
                         """) # Define the query - text() ensures that the query string is read as a SQL expression
    case_three = pd.read_sql(case_three, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
case_three

,count,salary_group
0,7423,5 Figures
1,91,6 Figures


**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Question:** Is there anyone in the database who does not have their driver's license?

In [14]:
with engine.connect() as connection: # Establish a connection
    null = text("""SELECT name
                   FROM person
                   WHERE license_id IS NULL
                   """) # Define the query - text() ensures that the query string is read as a SQL expression
    null = pd.read_sql(null, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
null

,name


There were no results, so everyone who is in the database has their driver's license.

**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Add my own name into the 'persons' table.
<br>
**What table(s) are we adding a record to?** person

In [16]:
with engine.connect() as connection: # Establish a connection
    connection.execute(text("""INSERT INTO person (name)
                               VALUES ('Thompson Morgan')
                               """)) # Insert into the 'person' table the specified value for the 'name' field

**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statements above.<br>
The IN operator in a WHERE clause lets you filter for multiple values at once. You can also exclude certain values by using the NOT IN operator.
<br>

**Question:** What is the highest and lowest annual income in the database?
<br>
**What table(s) are we joining? (If any)** None

In [18]:
with engine.connect() as connection: # Establish a connection
    min_max = text("""SELECT MIN(annual_income), MAX(annual_income)
                      FROM income
                      """) # Define the query - text() ensures that the query string is read as a SQL expression
    min_max = pd.read_sql(min_max, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
min_max

,MIN(annual_income),MAX(annual_income)
0,10000,498500


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What is the height of the tallest female/male, and the shortest female/male?
<br>
**What table(s) are we joining?** None

In [20]:
with engine.connect() as connection: # Establish a connection
    min_max_case = text("""SELECT MAX(CASE WHEN gender = 'female' THEN height END) AS tallest_female,
                            MAX(CASE WHEN gender = 'male' THEN height END) AS tallest_male,
                            MIN(CASE WHEN gender = 'female' THEN height END) AS shortest_female,
                            MIN(CASE WHEN gender = 'male' THEN height END) AS shortest_male
                           FROM drivers_license
                           """) # Define the query - text() ensures that the query string is read as a SQL expression
    min_max_case = pd.read_sql(min_max_case, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
min_max_case

,tallest_female,tallest_male,shortest_female,shortest_male
0,82,84,50,50


**MIN() and MAX()** <br>
AVG() will take the average of a numeric field.

**Question:** What is the average check-in time for the gym in our database?

In [22]:
with engine.connect() as connection: # Establish a connection
    average = text("""SELECT AVG(check_in_time)
                      FROM get_fit_now_check_in
                      """) # Define the query - text() ensures that the query string is read as a SQL expression
    average = pd.read_sql(average, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
average

,AVG(check_in_time)
0,600.537921


Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above. <br><br>
**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above (If you have not done so already). Be sure to obtain the same result set.

In [24]:
with engine.connect() as connection: # Establish a connection
    aliases = text("""SELECT COUNT(*) AS count,
                      CASE
                              WHEN height >= 84 THEN '7 feet tall or over'
                              WHEN height >= 72 THEN 'Between 6 and 7 feet tall'
                              WHEN height >= 60 THEN 'Between 5 and 6 feet tall'
                              WHEN height >= 48 THEN 'Between 4 and 5 feet tall'
                              ELSE 'Under 4 feet tall'
                          END AS height_group
                      FROM drivers_license AS d_l
                      GROUP BY height_group
                      ORDER BY count DESC
                      """) # Define the query - text() ensures that the query string is read as a SQL expression
    aliases = pd.read_sql(aliases, connection) # Use pandas to read the sql query with the connection to the database

# Print the results
aliases

,count,height_group
0,3613,Between 5 and 6 feet tall
1,3389,Between 6 and 7 feet tall
2,3004,Between 4 and 5 feet tall
3,1,7 feet tall or over


Now we are starting to create multiple new fields that we can save any time as a .csv if needed to access later. Save your result set as a .csv:

In [26]:
# Save result set as .csv file:
aliases.to_csv("test.csv", index=False)

**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Objective:** Delete my name from the database, after I added it above.
<br>
**What table(s) are we deleting records from?** person

In [28]:
with engine.connect() as connection: # Establish a connection
    delete = text("""DELETE FROM person
                     WHERE name = 'Thompson Morgan'
                     """) # Delete from the 'person' table the specified value for the 'name' field
    connection.execute(delete)
    connection.commit()

We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>

In [30]:
# Close the database connection :)
connection.close()

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. It is preferred that you submit your Github link instead of the file itself.